In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType, IntegerType, StringType, DateType, TimestampType

# Configurações do Spark
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.11.901')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')

# Criar sessão Spark
spark = SparkSession.builder.config(conf=conf).getOrCreate()

# Lista de arquivos CSV a serem lidos
csv_files = [
    'reviews-teste.csv'
]

# Prefixo do caminho S3
s3_prefix = 's3a://bucket-raw-upa-connect-otavio/'

# Criar a lista completa de caminhos S3
s3_paths = [s3_prefix + file for file in csv_files]

# Ler os CSVs necessários a partir da lista de caminhos
TabelaCompleta = spark.read.option('delimiter', ',') \
                             .option('header', 'true') \
                             .option('nullValue', 'null') \
                             .csv(s3_paths)
# TabelaCompleta.show()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0ff4f024-7236-4418-b9be-53691379a861;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 401ms :: artifacts dl 10ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 by [com.amazonaws#aws-java-sdk-bundle;1.12.262] in [default]
	---------------------------------------------------------------------
	|     

In [2]:
# Define a nova ordem e os novos nomes (tradução)
TabelaOrganizada = TabelaCompleta.select(
    col("title").alias("Nome_Upa"),
    col("reviewsCount").alias("Contagem_Avaliacoes"),
    col("totalScore").alias("Pontuacao_Total"),
    col("city").alias("Cidade"),
    col("state").alias("Estado"),
    col("neighborhood").alias("Bairro"),
    col("street").alias("Rua"),
    col("postalCode").alias("CEP"),
    col("reviewOrigin").alias("Origem_Avaliacao"),
    col("scrapedAt").alias("Data_Coleta"),
    col("publishedAtDate").alias("Data_Publicacao_Avaliacao"),
    col("text").alias("Texto_Avaliacao"),
    col("stars").alias("Estrelas")
)

# Exibe o novo DataFrame com as colunas reordenadas e traduzidas
# TabelaOrganizada.show()

In [3]:
from pyspark.sql.types import IntegerType, DoubleType
from pyspark.sql.functions import col, to_timestamp, date_format

TabelaTipada = TabelaOrganizada.withColumn(
    # Convertendo colunas numéricas
    "Contagem_Avaliacoes", col("Contagem_Avaliacoes").cast(IntegerType())
).withColumn(
    "Pontuacao_Total", col("Pontuacao_Total").cast(DoubleType())
).withColumn(
    "Estrelas", col("Estrelas").cast(IntegerType())
).withColumn(
    # Convertendo e formatando as colunas de data para string no padrão ISO 8601
    "Data_Coleta", 
    date_format(to_timestamp(col("Data_Coleta")), "yyyy-MM-dd'T'HH:mm:ss")
).withColumn(
    "Data_Publicacao_Avaliacao", 
    date_format(to_timestamp(col("Data_Publicacao_Avaliacao")), "yyyy-MM-dd'T'HH:mm:ss")
)

# Verifica o resultado da transformação
print("Schema após a conversão de tipos e formatação de datas:")
TabelaTipada.printSchema()

# print("\nDados com os tipos corrigidos e datas em formato ISO 8601:")
# TabelaTipada.show(truncate=False)

Schema após a conversão de tipos e formatação de datas:
root
 |-- Nome_Upa: string (nullable = true)
 |-- Contagem_Avaliacoes: integer (nullable = true)
 |-- Pontuacao_Total: double (nullable = true)
 |-- Cidade: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Rua: string (nullable = true)
 |-- CEP: string (nullable = true)
 |-- Origem_Avaliacao: string (nullable = true)
 |-- Data_Coleta: string (nullable = true)
 |-- Data_Publicacao_Avaliacao: string (nullable = true)
 |-- Texto_Avaliacao: string (nullable = true)
 |-- Estrelas: integer (nullable = true)



In [4]:
# Preencher valores nulos na coluna de texto com uma string padrão
TabelaSemNulos = TabelaTipada.fillna({"Texto_Avaliacao": "Avaliação não preenchida"})

print("Dados após tratamento de nulos:")
TabelaSemNulos.show()

Dados após tratamento de nulos:
+---------------+-------------------+---------------+---------+---------+-----------+--------------------+---------+----------------+-------------------+-------------------------+--------------------+--------+
|       Nome_Upa|Contagem_Avaliacoes|Pontuacao_Total|   Cidade|   Estado|     Bairro|                 Rua|      CEP|Origem_Avaliacao|        Data_Coleta|Data_Publicacao_Avaliacao|     Texto_Avaliacao|Estrelas|
+---------------+-------------------+---------------+---------+---------+-----------+--------------------+---------+----------------+-------------------+-------------------------+--------------------+--------+
|UPA Santo Amaro|               1027|            3.0|São Paulo|São Paulo|Santo Amaro|R. Promotor Gabri...|04743-050|          Google|2025-10-18T02:18:50|      2025-10-17T22:53:49|Pecismo atendido ...|       1|
|UPA Santo Amaro|               1027|            3.0|São Paulo|São Paulo|Santo Amaro|R. Promotor Gabri...|04743-050|          Go

In [5]:
import unicodedata
from pyspark.sql.functions import col, upper, udf
from pyspark.sql.types import StringType

# 1. Criar a função Python que remove os acentos
def remover_acentos(texto):
    if texto is None:
        return None
    # Normaliza o texto para o formato 'NFKD', que separa os caracteres de seus acentos
    nfkd_form = unicodedata.normalize('NFKD', texto)
    # Codifica para ASCII ignorando os caracteres que não consegue converter (os acentos)
    # e depois decodifica de volta para UTF-8
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

# 2. Registrar a função Python como uma UDF do Spark
remover_acentos_udf = udf(remover_acentos, StringType()) 

# Lista das colunas que precisam de tratamento
colunas_texto = [
    "Nome_Upa",
    "Cidade",
    "Estado",
    "Bairro",
    "Rua",
    "Texto_Avaliacao"
]

# 3. Aplica a transformação em um loop
for nome_coluna in colunas_texto:
    TabelaSemNulos = TabelaSemNulos.withColumn(
        nome_coluna,
        # Aplica primeiro a remoção de acentos e depois converte para caixa alta
        upper(remover_acentos_udf(col(nome_coluna)))
    )

# Exibe o resultado para verificação
print("DataFrame com texto padronizado (sem acentos e em caixa alta):")
TabelaSemNulos.show()

DataFrame com texto padronizado (sem acentos e em caixa alta):


+---------------+-------------------+---------------+---------+---------+-----------+--------------------+---------+----------------+-------------------+-------------------------+--------------------+--------+
|       Nome_Upa|Contagem_Avaliacoes|Pontuacao_Total|   Cidade|   Estado|     Bairro|                 Rua|      CEP|Origem_Avaliacao|        Data_Coleta|Data_Publicacao_Avaliacao|     Texto_Avaliacao|Estrelas|
+---------------+-------------------+---------------+---------+---------+-----------+--------------------+---------+----------------+-------------------+-------------------------+--------------------+--------+
|UPA SANTO AMARO|               1027|            3.0|SAO PAULO|SAO PAULO|SANTO AMARO|R. PROMOTOR GABRI...|04743-050|          Google|2025-10-18T02:18:50|      2025-10-17T22:53:49|PECISMO ATENDIDO ...|       1|
|UPA SANTO AMARO|               1027|            3.0|SAO PAULO|SAO PAULO|SANTO AMARO|R. PROMOTOR GABRI...|04743-050|          Google|2025-10-18T02:18:50|      2

In [8]:
from pyspark.sql.functions import col, when, regexp_replace, trim
from functools import reduce

# DataFrame de entrada
df_entrada = TabelaSemNulos

# --- Passo 1: Definir a lógica de substituição completa (casos específicos) ---
# A coluna 'Nome_Upa' é avaliada e, se corresponder a uma condição, é substituída.
# Caso contrário, o valor original é mantido com .otherwise().
coluna_com_substituicoes = (
    when(col("Nome_Upa") == "PRONTO SOCORRO MUNICIPAL DA LAPA", "UPA LAPA - PROF. JOÃO CATARIN MEZOMO")
    .when(col("Nome_Upa") == "UPA MOOCA", "UPA MOOCA - DOM PAULO EVARISTO ARNS")
    .when(col("Nome_Upa") == "UPA PERUS", "UPA PERUS - DR. LUIZ ANTONIO DE ABREU SAMPAIO DORIA")
    .when(col("Nome_Upa") == "UPA SANTO AMARO", "UPA SANTO AMARO - DR JOSÉ SYLVIO DE CAMARGO")
    .when(col("Nome_Upa") == "UPA TATUAPE", "UPA TATUAPE - WALDEMAR ROSSI")
    .when(col("Nome_Upa") == "UPA VILA MARIA", "UPA VILA MARIA - DR JOSE MAURO DEL ROIO CORREA")
    .when(col("Nome_Upa") == "UPA VILA MARIANA - UNIDADE DE PRONTO ATENDIMENTO", "UPA VILA MARIANA")
    .when(col("Nome_Upa") == "UPA - CAMPO LIMPO", "UPA CAMPO LIMPO - DR FERNANDO MAURO PROENÇA DE GOUVEA")
    .when(col("Nome_Upa").contains("AMA / UBS INTEGRADA"), "UPA JARDIM ICARAI QUINTANA")
    .otherwise(col("Nome_Upa"))
)

# --- Passo 2: Definir a lista de regras de limpeza (substituições parciais) ---
# Cada tupla contém: (o texto a ser encontrado, o texto que o substituirá)
substituicoes_parciais = [
    (" III", ""),
    ("UPA -", "UPA"),
    ("/UBS", ""),
    ("CARRAO - ", "CARRAO "),
    (" - SANTA MARCELINA", ""),
    (" - EMERGENCY CARE UNIT", "")
]

# --- Passo 3: Usar 'reduce' para aplicar todas as regras de limpeza em sequência ---
# 'reduce' itera sobre a lista 'substituicoes_parciais', aplicando a função regexp_replace
# cumulativamente, começando com o resultado do Passo 1.
coluna_final_tratada = reduce(
    lambda c, regra: regexp_replace(c, regra[0], regra[1]),
    substituicoes_parciais,
    coluna_com_substituicoes
)

# --- Passo 4: Aplicar a transformação final ao DataFrame ---
# Sobrescrevemos a coluna "Nome_Upa" com o resultado de todas as regras,
# aplicando o trim() para remover espaços extras no início e no fim.
TabelaFinal = df_entrada.withColumn(
    "Nome_Upa", 
    trim(coluna_final_tratada)
)

# Ordena o DataFrame pela coluna 'Nome_Upa' em ordem ascendente (A-Z)
TabelaFinal = TabelaFinal.orderBy(col("Nome_Upa").asc())

# --- Validação: Mostrar o resultado para conferência ---
print("Nomes das UPAs ANTES da padronização:")
df_entrada.select("Nome_Upa").distinct().show(n=100, truncate=False)

print("\nNomes das UPAs DEPOIS da padronização:")
TabelaFinal.select("Nome_Upa").distinct().show(n=100, truncate=False)

TabelaFinal.show()

Nomes das UPAs ANTES da padronização:
+------------------------------------------------+
|Nome_Upa                                        |
+------------------------------------------------+
|UPA - PARELHEIROS - EMERGENCY CARE UNIT         |
|PRONTO SOCORRO MUNICIPAL DA LAPA                |
|UPA MOOCA                                       |
|UPA RIO PEQUENO                                 |
|UPA PIRITUBA                                    |
|UPA - CAMPO LIMPO                               |
|UPA JARDIM HELENA - SANTA MARCELINA             |
|UPA - CITY JARAGUA                              |
|UPA/UBS JARDIM ELISA MARIA I                    |
|UPA - ERMELINO MATARAZZO                        |
|AMA / UBS INTEGRADA JARDIM ICARAI QUINTANA      |
|UPA SANTO AMARO                                 |
|UPA III - JARDIM PERI                           |
|UPA III CARRAO - MASATAKA OTA                   |
|UPA PERUS                                       |
|UPA 26 DE AGOSTO                           

In [7]:
# 1. Definir o caminho de saída no bucket "trusted"
# É uma boa prática salvar em um subdiretório para manter a organização.
caminho_saida_s3 = "s3a://bucket-trusted-upa-connect-otavio/avaliacoes-google-tratadas"

print(f"Iniciando a gravação do arquivo CSV em: {caminho_saida_s3}")

# 2. Gravar o DataFrame no S3 como um único arquivo CSV
TabelaFinal.coalesce(1) \
    .write \
    .mode("overwrite") \
    .option("header", "true") \
    .option("delimiter", ";") \
    .csv(caminho_saida_s3)

print("Gravação concluída com sucesso!")

Iniciando a gravação do arquivo CSV em: s3a://bucket-trusted-upa-connect-otavio/avaliacoes-google-tratadas


25/10/18 19:00:52 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
25/10/18 19:00:53 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


Gravação concluída com sucesso!
